# Azure OpenAI Embeddings デモ

このNotebookでは Azure OpenAI Service の **Embeddings API** を用いてテキストを数値ベクトル（高次元特徴量）へ変換し、類似度計算・検索・分類・FAQマッチングなどの応用例を段階的に紹介します。

### 🔰 Embeddings API 概要
Embeddings はテキスト（単語 / 文 / 段落 / ドキュメント）を **意味的距離を反映した固定次元ベクトル** に変換します。これにより以下が可能になります:
- 類似度検索（Semantic Search）
- クラスタリング / 分類
- レコメンデーション
- RAG 前段のフィルタリング
- FAQ マッチング / 重複検出

### 🧬 利用可能な代表モデル（2025年時点 Azure OpenAI）
| モデル | 次元数例 | 主用途 | 特徴 |
|--------|----------|--------|------|
| `text-embedding-3-large` | 3,072 | 高精度検索・分類 | 高次元・精度最重視 |
| `text-embedding-3-small` | 1,536 | 一般用途/バランス | コスト/性能バランス良好 |
| `text-embedding-ada-002` | 1,536 | 既存互換利用 | 旧世代。新規は3系推奨 |
| `gpt-4o-mini-embeddings` | 1,536 | マルチ用途統合 | GPT-4o miniと連携設計 |

> 新規開発は通常 **`text-embedding-3-small` → 精度要件が厳しければ `text-embedding-3-large`** の順で検討します。コスト最適化が重要な大量バッチ用途では small を推奨。

### 🧪 このNotebookで扱う主な概念
1. 基本的な埋め込み取得
2. コサイン類似度計算
3. Semantic Search の最小実装
4. カテゴリ分類（ゼロショット的）
5. FAQマッチング例
6. バッチ埋め込み最適化
7. 応用とベストプラクティス

以降の各コードセル前に、実行する処理の概要を数行で説明します。

## 1. 環境セットアップ

### 環境変数の設定

リポジトリルートに `.env.dev` ファイルを作成し、以下の環境変数を設定してください：

```bash
# PowerShell の場合
@"
AZURE_OPENAI_ENDPOINT=https://your-resource.openai.azure.com/
AZURE_OPENAI_API_KEY=your-api-key-here
AZURE_OPENAI_API_VERSION=2024-08-01-preview
EMBEDDING_DEPLOYMENT_NAME=text-embedding-ada-002
"@ | Out-File -FilePath ..\..\..env.dev -Encoding utf8
```

または、手動で `.env.dev` ファイルを作成してください。

**注意**: `.env.dev` ファイルは `.gitignore` に含まれており、Git にコミットされません。

---
**補足1**: `.env.dev` の存在確認とテンプレート生成を行います。既にある場合は再生成せずユーザーに編集を促します。APIキー管理の初期準備ステップです。

**補足2**: リポジトリルートの `.env.dev` を強制パースし、埋め込み API 呼び出しに必要なエンドポイントやキーを `os.environ` に設定します。空値は無視し、秘密情報は出力しません。

**補足3**: Azure OpenAI クライアントを初期化し、使用する Embeddings モデル名を表示します。接続設定が正しく行われているか確認するフェーズです。

In [ ]:
# 必要なライブラリのインストール確認
import sys
print(f"Python バージョン: {sys.version}")

try:
    import openai
    print(f"✅ openai バージョン: {openai.__version__}")
except ImportError:
    print("❌ openai ライブラリがインストールされていません")
    print("インストール: pip install openai")

try:
    import numpy
    print(f"✅ numpy バージョン: {numpy.__version__}")
except ImportError:
    print("❌ numpy ライブラリがインストールされていません")
    print("インストール: pip install numpy")

In [ ]:
# .env.dev を強制パースして os.environ に直接設定する
from dotenv import dotenv_values
from pathlib import Path
import os

# ノートブックから見たリポジトリルートへの相対パス (demos/openai から ../../ でルートへ)
repo_root = Path(__file__).parent.parent.parent if '__file__' in globals() else Path.cwd().parent.parent
env_path = repo_root / '.env.dev'

if env_path.exists():
    # dotenv_values で .env.dev を辞書として読み込む
    parsed = dotenv_values(env_path)
    # 各キーを os.environ に書き込む(空文字や None はスキップ)
    for k, v in parsed.items():
        if v:
            os.environ[k] = v
    print(f'環境変数を読み込みました: {env_path}')
    print(f'{len(parsed)} 個のキーをパースし、os.environ に設定しました。')
else:
    print(f'警告: {env_path} が見つかりません。')

# 設定状況を確認
print('\n設定状況:')
print('  AZURE_OPENAI_ENDPOINT:', 'SET' if os.getenv('AZURE_OPENAI_ENDPOINT') else 'NOT SET')
print('  AZURE_OPENAI_API_KEY:', 'SET' if os.getenv('AZURE_OPENAI_API_KEY') else 'NOT SET')
print('  EMBEDDING_DEPLOYMENT_NAME:', os.getenv('EMBEDDING_DEPLOYMENT_NAME', 'NOT SET'))

In [ ]:
# Azure OpenAI クライアントの初期化
from openai import AzureOpenAI
import os

# 環境変数から設定を取得
endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
api_key = os.getenv("AZURE_OPENAI_API_KEY")
api_version = os.getenv("AZURE_OPENAI_API_VERSION", "2024-08-01-preview")
embedding_deployment = os.getenv("EMBEDDING_DEPLOYMENT_NAME", "text-embedding-ada-002")

# クライアントの初期化
try:
    client = AzureOpenAI(
        api_key=api_key,
        api_version=api_version,
        azure_endpoint=endpoint
    )
    print("✅ Azure OpenAI クライアントが正常に初期化されました")
    print(f"エンドポイント: {endpoint}")
    print(f"API バージョン: {api_version}")
    print(f"Embedding デプロイ名: {embedding_deployment}")
except Exception as e:
    print(f"❌ クライアントの初期化に失敗しました: {e}")

## 2. 基本的な埋め込み生成

**概要**: 単一テキストを埋め込みベクトルに変換する基本関数 `get_embedding` を定義します。戻り値は数値配列 (List[float]) で後続の類似度計算や検索、分類に利用します。埋め込みベクトルの次元数を確認し、先頭要素を抜粋表示して形状を把握します。

In [ ]:
def get_embedding(text: str, model: str = None) -> list[float]:
    """
    テキストの埋め込みベクトルを取得する
    
    Args:
        text: 埋め込みを生成するテキスト
        model: 使用するモデル名（デフォルトは環境変数から取得）
        
    Returns:
        埋め込みベクトル（floatのリスト）
    """
    if model is None:
        model = embedding_deployment
    
    try:
        response = client.embeddings.create(
            input=text,
            model=model
        )
        return response.data[0].embedding
    except Exception as e:
        print(f"エラーが発生しました: {e}")
        return None

# デモ実行
sample_text = "Azure AI Services は Microsoft が提供するクラウドベースの AI サービスです。"
embedding = get_embedding(sample_text)

if embedding:
    print(f"テキスト: {sample_text}")
    print(f"\n埋め込みベクトルの次元数: {len(embedding)}")
    print(f"最初の10要素: {embedding[:10]}")

## 3. コサイン類似度の計算

**概要**: 埋め込み同士の意味的近さを測るためにコサイン類似度関数 `cosine_similarity` を定義します。値域は -1〜1 で 1 に近いほど高い類似度です。複数テキスト間で比較し、近いペアと遠いペアのスコア差を確認します。

In [ ]:
import numpy as np

def cosine_similarity(vec1: list[float], vec2: list[float]) -> float:
    """
    2つのベクトル間のコサイン類似度を計算する
    
    Args:
        vec1: ベクトル1
        vec2: ベクトル2
        
    Returns:
        コサイン類似度（-1 から 1 の範囲）
    """
    vec1 = np.array(vec1)
    vec2 = np.array(vec2)
    
    dot_product = np.dot(vec1, vec2)
    norm1 = np.linalg.norm(vec1)
    norm2 = np.linalg.norm(vec2)
    
    return dot_product / (norm1 * norm2)

# 類似度計算のデモ
text1 = "犬は人間の最良の友です。"
text2 = "犬は忠実なペットです。"
text3 = "Python はプログラミング言語です。"

embedding1 = get_embedding(text1)
embedding2 = get_embedding(text2)
embedding3 = get_embedding(text3)

if all([embedding1, embedding2, embedding3]):
    similarity_1_2 = cosine_similarity(embedding1, embedding2)
    similarity_1_3 = cosine_similarity(embedding1, embedding3)
    similarity_2_3 = cosine_similarity(embedding2, embedding3)
    
    print(f"テキスト1: {text1}")
    print(f"テキスト2: {text2}")
    print(f"テキスト3: {text3}\n")
    
    print(f"類似度（テキスト1 vs テキスト2）: {similarity_1_2:.4f}")
    print(f"類似度（テキスト1 vs テキスト3）: {similarity_1_3:.4f}")
    print(f"類似度（テキスト2 vs テキスト3）: {similarity_2_3:.4f}")
    
    print("\n💡 解釈: 値が 1 に近いほど類似、0 に近いほど無関係")

## 4. セマンティック検索のデモ

**概要**: コーパス（ドキュメント集合）の埋め込みを事前計算し、クエリ埋め込みとのコサイン類似度で上位候補を取得する最小実装 `semantic_search` を示します。スコアの高い順に並べることで意味的検索が成立します。

In [ ]:
# サンプルドキュメント（実際のアプリケーションではデータベースから取得）
documents = [
    "Azure OpenAI Service は、OpenAI の強力な言語モデルを Azure で利用できるサービスです。",
    "Azure Computer Vision は、画像から情報を抽出する AI サービスです。",
    "Azure Speech Services は、音声認識と音声合成を提供します。",
    "Azure Language Services は、テキスト分析や言語理解を可能にします。",
    "Azure AI Services は、さまざまな AI 機能を提供する統合プラットフォームです。",
    "機械学習モデルのトレーニングには大量のデータが必要です。",
    "深層学習は、ニューラルネットワークを使用した機械学習の一種です。",
    "自然言語処理（NLP）は、人間の言語をコンピュータが理解する技術です。",
]

# 各ドキュメントの埋め込みを生成
print("ドキュメントの埋め込みを生成中...")
doc_embeddings = []
for i, doc in enumerate(documents):
    embedding = get_embedding(doc)
    if embedding:
        doc_embeddings.append(embedding)
        print(f"✅ ドキュメント {i+1}/{len(documents)} 完了")

print(f"\n合計 {len(doc_embeddings)} 件のドキュメント埋め込みを生成しました")

In [ ]:
def semantic_search(query: str, documents: list[str], doc_embeddings: list[list[float]], top_k: int = 3) -> list[tuple[int, float, str]]:
    """
    セマンティック検索を実行する
    
    Args:
        query: 検索クエリ
        documents: ドキュメントのリスト
        doc_embeddings: ドキュメントの埋め込みベクトルのリスト
        top_k: 返す結果の数
        
    Returns:
        (インデックス, 類似度スコア, ドキュメント) のタプルのリスト
    """
    # クエリの埋め込みを生成
    query_embedding = get_embedding(query)
    if not query_embedding:
        return []
    
    # 各ドキュメントとの類似度を計算
    similarities = []
    for i, doc_emb in enumerate(doc_embeddings):
        similarity = cosine_similarity(query_embedding, doc_emb)
        similarities.append((i, similarity, documents[i]))
    
    # 類似度でソートして上位k件を返す
    similarities.sort(key=lambda x: x[1], reverse=True)
    return similarities[:top_k]

# セマンティック検索のデモ
queries = [
    "テキストを理解する AI サービスは？",
    "画像認識について教えてください",
    "音声に関する技術は？"
]

for query in queries:
    print(f"\n{'='*60}")
    print(f"🔍 クエリ: {query}")
    print(f"{'='*60}")
    
    results = semantic_search(query, documents, doc_embeddings, top_k=3)
    
    for rank, (idx, score, doc) in enumerate(results, 1):
        print(f"\n{rank}位 (類似度: {score:.4f})")
        print(f"  {doc}")

## 5. テキスト分類のデモ

**概要**: カテゴリ代表文（説明）の埋め込みを辞書として保持し、入力文の埋め込みとの類似度で最も近いカテゴリを推定する簡易ゼロショット分類です。スコア閾値を導入すれば“不明”カテゴリを設けることも可能です。

In [ ]:
# カテゴリとその説明
categories = {
    "技術": "プログラミング、AI、機械学習、ソフトウェア開発に関するトピック",
    "ビジネス": "経営、マーケティング、セールス、企業戦略に関するトピック",
    "科学": "物理学、化学、生物学、自然科学に関するトピック",
    "スポーツ": "サッカー、野球、オリンピック、フィットネスに関するトピック",
    "料理": "レシピ、食材、調理法、グルメに関するトピック"
}

# カテゴリ説明の埋め込みを生成
print("カテゴリの埋め込みを生成中...")
category_embeddings = {}
for category, description in categories.items():
    embedding = get_embedding(description)
    if embedding:
        category_embeddings[category] = embedding
        print(f"✅ {category} 完了")

print(f"\n{len(category_embeddings)} 個のカテゴリ埋め込みを生成しました")

In [ ]:
def classify_text(text: str, category_embeddings: dict[str, list[float]]) -> list[tuple[str, float]]:
    """
    テキストを分類する
    
    Args:
        text: 分類するテキスト
        category_embeddings: カテゴリ名と埋め込みベクトルの辞書
        
    Returns:
        (カテゴリ名, 類似度スコア) のタプルのリスト（スコア順）
    """
    # テキストの埋め込みを生成
    text_embedding = get_embedding(text)
    if not text_embedding:
        return []
    
    # 各カテゴリとの類似度を計算
    scores = []
    for category, cat_embedding in category_embeddings.items():
        similarity = cosine_similarity(text_embedding, cat_embedding)
        scores.append((category, similarity))
    
    # スコア順にソート
    scores.sort(key=lambda x: x[1], reverse=True)
    return scores

# テキスト分類のデモ
test_texts = [
    "Python で機械学習モデルを実装する方法について学んでいます。",
    "今日の夕食にパスタを作りました。トマトソースが美味しかったです。",
    "ワールドカップの決勝戦は素晴らしい試合でした。",
    "新製品のマーケティング戦略を立案する必要があります。",
    "量子力学の基礎原理について研究しています。"
]

for text in test_texts:
    print(f"\n{'='*60}")
    print(f"📝 テキスト: {text}")
    print(f"{'='*60}")
    
    results = classify_text(text, category_embeddings)
    
    print("\n分類結果:")
    for rank, (category, score) in enumerate(results, 1):
        bar_length = int(score * 50)  # スコアを棒グラフで表示
        bar = '█' * bar_length + '░' * (50 - bar_length)
        print(f"{rank}. {category:10s} {score:.4f} {bar}")

## 6. バッチ処理（複数テキストの一括埋め込み）

**概要**: 一度のAPI呼び出しで複数テキストを埋め込み取得する `get_embeddings_batch` により、ループ回数とネットワーク往復を削減しスループット・コストを最適化します。個別取得との次元・値の整合性を確認し、パフォーマンス改善の第一歩を示します。

In [ ]:
def get_embeddings_batch(texts: list[str], model: str = None) -> list[list[float]]:
    """
    複数のテキストの埋め込みを一括取得する
    
    Args:
        texts: テキストのリスト
        model: 使用するモデル名
        
    Returns:
        埋め込みベクトルのリスト
    """
    if model is None:
        model = embedding_deployment
    
    try:
        response = client.embeddings.create(
            input=texts,
            model=model
        )
        return [item.embedding for item in response.data]
    except Exception as e:
        print(f"エラーが発生しました: {e}")
        return []

# バッチ処理のデモ
batch_texts = [
    "Azure は Microsoft のクラウドプラットフォームです。",
    "AI と機械学習はビジネスを変革しています。",
    "クラウドコンピューティングは現代の IT インフラの基盤です。"
]

print("バッチ処理で埋め込みを生成中...")
batch_embeddings = get_embeddings_batch(batch_texts)

if batch_embeddings:
    print(f"\n✅ {len(batch_embeddings)} 件の埋め込みを生成しました")
    for i, (text, embedding) in enumerate(zip(batch_texts, batch_embeddings), 1):
        print(f"\n{i}. {text}")
        print(f"   次元数: {len(embedding)}")

## 7. 実践的な使用例: FAQシステム

**概要**: FAQエントリ質問文の埋め込みをインデックスし、ユーザー質問埋め込みとの類似度で最良候補を選択します。さらに閾値付き関数 `find_best_faq` で関連性が低い場合の fallback（未回答ハンドリング）を追加し、実サービス適用時の品質確保パターンを示します。

In [ ]:
# FAQ データベース
faq_data = [
    {
        "question": "Azure OpenAI Service とは何ですか？",
        "answer": "Azure OpenAI Service は、OpenAI の強力な言語モデル（GPT-4、GPT-3.5、DALL-E など）を Azure のセキュリティとエンタープライズ機能と組み合わせたサービスです。"
    },
    {
        "question": "料金はどのように計算されますか？",
        "answer": "Azure OpenAI Service の料金は、使用したトークン数に基づいて計算されます。モデルやリージョンによって価格が異なります。詳細は Azure の料金ページをご確認ください。"
    },
    {
        "question": "どのリージョンで利用できますか？",
        "answer": "Azure OpenAI Service は、East US、West Europe、Japan East など、複数の Azure リージョンで利用可能です。最新の利用可能リージョンは公式ドキュメントをご確認ください。"
    },
    {
        "question": "API の呼び出し制限はありますか？",
        "answer": "はい、レート制限があります。デプロイメントごとに TPM（Tokens Per Minute）と RPM（Requests Per Minute）の制限が設定されています。これらの制限は Azure Portal で調整可能です。"
    },
    {
        "question": "データのセキュリティは保証されていますか？",
        "answer": "はい、Azure OpenAI Service は Azure のセキュリティ機能を継承しており、データの暗号化、プライベートエンドポイント、RBAC などのセキュリティ機能が利用できます。"
    }
]

# FAQ の質問部分の埋め込みを生成
print("FAQ データの埋め込みを生成中...")
faq_questions = [item["question"] for item in faq_data]
faq_embeddings = get_embeddings_batch(faq_questions)

if faq_embeddings:
    print(f"✅ {len(faq_embeddings)} 件の FAQ 埋め込みを生成しました\n")

In [ ]:
def find_best_faq(user_question: str, faq_data: list[dict], faq_embeddings: list[list[float]], threshold: float = 0.7) -> tuple[dict, float] | None:
    """
    ユーザーの質問に最も近い FAQ を検索する
    
    Args:
        user_question: ユーザーの質問
        faq_data: FAQ データのリスト
        faq_embeddings: FAQ の埋め込みベクトルのリスト
        threshold: 類似度の閾値（これ以上のスコアの場合のみ返す）
        
    Returns:
        (FAQ アイテム, 類似度スコア) のタプル、または None
    """
    # ユーザー質問の埋め込みを生成
    question_embedding = get_embedding(user_question)
    if not question_embedding:
        return None
    
    # 最も類似度の高い FAQ を検索
    best_score = -1
    best_faq = None
    
    for faq_item, faq_emb in zip(faq_data, faq_embeddings):
        similarity = cosine_similarity(question_embedding, faq_emb)
        if similarity > best_score:
            best_score = similarity
            best_faq = faq_item
    
    # 閾値チェック
    if best_score >= threshold:
        return best_faq, best_score
    else:
        return None

# FAQ システムのデモ
user_questions = [
    "Azure OpenAI とは？",
    "費用はいくらかかりますか？",
    "日本で使えますか？",
    "セキュリティは大丈夫？",
    "天気はどうですか？"  # 関連性の低い質問
]

for question in user_questions:
    print(f"\n{'='*60}")
    print(f"❓ ユーザーの質問: {question}")
    print(f"{'='*60}")
    
    result = find_best_faq(question, faq_data, faq_embeddings, threshold=0.7)
    
    if result:
        faq_item, score = result
        print(f"\n✅ マッチした FAQ (類似度: {score:.4f}):")
        print(f"Q: {faq_item['question']}")
        print(f"\nA: {faq_item['answer']}")
    else:
        print("\n❌ 関連する FAQ が見つかりませんでした。")
        print("カスタマーサポートにお問い合わせください。")

## まとめ

このノートブックでは、Azure OpenAI の Embeddings API を使用して以下を実装しました：

### 実装した機能
1. ✅ テキストの埋め込みベクトル生成
2. ✅ コサイン類似度の計算
3. ✅ セマンティック検索
4. ✅ テキスト分類
5. ✅ バッチ処理
6. ✅ FAQ システム（実践例）

### 活用例
- **検索システム**: キーワードマッチではなく意味的な検索
- **推薦システム**: 類似コンテンツの推薦
- **分類システム**: ドキュメントやメッセージの自動分類
- **重複検出**: 類似テキストの検出
- **クラスタリング**: テキストのグループ化

### ベストプラクティス
- バッチ処理を活用してコストと時間を削減
- 埋め込みベクトルをキャッシュして再利用
- 適切な閾値を設定して精度を向上
- エラーハンドリングを実装

### 次のステップ
- ベクトルデータベース（Azure Cosmos DB, Azure Cognitive Search, Pinecone など）との統合
- RAG（Retrieval-Augmented Generation）パターンの実装
- より大規模なデータセットでの実験

---
**補足**: Cosmos DB などを活用する際はパーティションキー設計と埋め込みキャッシュ戦略（事前計算・更新頻度管理）を行い、ホットパーティション回避とクエリ効率化を図るとスケールが容易になります。